# Scraper
-----
- Load most recent weather.csv
- Collect list of missing data between start_date and today
- Scrape balloon data for any missing dates
- Clean / normalize new data
- Concatenate new and histrolcal DataFrames
- Update weather.csv
---

In [261]:
import os
import requests
import pandas as pd
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import numpy as np
from math import floor
from pathlib import Path  
from alive_progress import alive_bar
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
from windrose import WindroseAxes
from PIL import Image

## Load Historical Weather Data
---

In [262]:
data_path = './data/weather.csv'

# If data exists, load it
# ~~~~~~~~~~~~~~~~~~~~~~~
if os.path.exists(data_path):
    weather_data = pd.read_csv(data_path)
    print(weather_data.head())
else:
    print("No historical data exists")

   Unnamed: 0        Date     Station  Pressure  Height  Temp  DewPoint  \
0           0  2020-01-01  Quillayute       700  2969.0  -0.9      -0.9   
1           1  2020-01-01  Quillayute       850  1393.0   6.2       6.2   
2           2  2020-01-01      Vernon       700  2880.0  -5.5      -7.0   
3           3  2020-01-01      Vernon       850  1357.0  -3.9      -5.8   
4           4  2020-01-01  Port Hardy       700  2856.0  -4.3      -5.8   

   Relative_Humidity  Mean_Mixed_Layer  Wind_Direction  Wind_Speed  \
0              100.0              5.14           255.0     100.008   
1              100.0              7.04           260.0      81.488   
2               89.0              3.25           260.0      94.452   
3               87.0              2.93           170.0      18.520   
4               89.0              3.56           250.0      87.044   

   Potential_Temp  Equivalent_Potential_Temp  Virtual_Potential_Temp  
0           301.5                      317.2             

## Create Range of Missing Dates
---
Extension: Check any dates that are incomplete
- Missing one location but not all
- Missing values

In [263]:
dates = []
delta = timedelta(days=1)
start_date = date(2020, 1, 1)
end_date = datetime.today().date()

# Create list of all dates between start_date and today
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
while start_date <= end_date:
    # add current date to list by converting  it to iso format
    dates.append(start_date.isoformat())
    # increment start date by timedelta
    start_date += delta

In [264]:
# Display start and end of date range
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for x in range(5):
    print(dates[x])
print('...')   
for x in range(5):
    print(dates[-x])

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
...
2020-01-01
2023-06-21
2023-06-20
2023-06-19
2023-06-18


In [265]:
removed = []
# Remove dates from list that are alrady populated in weather_data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for d in weather_data['Date'].unique().tolist():
    if d in dates:
        dates.remove(d)
        removed.append(d)

### Extension
would it also be worth checking for entries with missing values?

In [266]:
# Display start and end of date range
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for d in removed:
    print(d)

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31

## Generate URLs to be Scraped
---

In [267]:
base_url = "http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST"

stations = [
    {"id": 72797, "name": "Quillayute"},
    {"id": 73033, "name": "Vernon"},
    {"id": 71109, "name": "Port Hardy"}]

In [268]:
for d in dates:
    print(d)

2021-12-16
2023-06-20
2023-06-21


In [269]:
# Create list of URLs for each station
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
urls = []
dates_idx = []
station_idx = []
for d in dates:
    d = d.split('-')
    for station in stations:
        url = base_url
        url += "&YEAR={}".format(d[0])
        url += "&MONTH={}".format(d[1])
        #url += "&FROM=" + str(now[1]) + str(now[2])
        #url += "&TO=" + str(now[1]) + str(now[2])
        url += "&FROM={:0>2d}00".format(int(d[2]))
        url += "&TO={:0>2d}23".format(int(d[2]))
        url += "&STNM={}".format(station["id"])
        url += "&REPLOT=1"
        urls.append(url)
        
        
        dates_idx.append("%02d-%02d-%02d" % (int(d[0]), int(d[1]), int(d[2])))
        station_idx.append(station["name"])


In [270]:
for url in urls:
    print(url)

http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=12&FROM=1600&TO=1623&STNM=72797&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=12&FROM=1600&TO=1623&STNM=73033&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=12&FROM=1600&TO=1623&STNM=71109&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2000&TO=2023&STNM=72797&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2000&TO=2023&STNM=73033&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2000&TO=2023&STNM=71109&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2100&TO=2123&STNM=72797&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2

## Extract data from list of URLs
---
- Iterate through URLs
- Extract 700 and 850 data items
- Append data to array

In [271]:
data = []
reading_idx = []
problem_urls = []

print("Extracting data from URLs: ")
with alive_bar(len(urls), force_tty=True) as bar:
    for i, url in enumerate(urls):
        search = 0
        while search >= 0:
            # Scrape site data
            # ~~~~~~~~~~~~~~~~
            search += 1
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            results = str(soup.find('pre'))
            sevhundy = results[results.find("700"):].split()[:11]
            eightfiddy = results[results.find("850"):].split()[:11]
            
            # Catch erroneous reads
            # Site can hit too many requests
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            if 'e' not in sevhundy[0] and 'e' not in eightfiddy[0]:
                search = -1
            elif search == 10:
                print(url)
                problem_urls.append(url)
                search = -1

        # Extract 700 first // 850 second
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        data.append(sevhundy)
        data[-1].insert(0, dates_idx[i])
        data[-1].insert(1, station_idx[i])
        
        data.append(eightfiddy)
        data[-1].insert(0, dates_idx[i])
        data[-1].insert(1, station_idx[i])
        
        bar()
        

print("\nComplete")

Extracting data from URLs: 
|████████████████████████████████████████| 9/9 [100%] in 20.6s (0.41/s)                                                 

Complete


In [272]:
for row in data:
    print(row)

['2021-12-16', 'Quillayute', '700.0', '2844', '-10.9', '-11.3', '97', '2.31', '125', '16', '290.4', '297.4', '290.8']
['2021-12-16', 'Quillayute', '850.0', '1328', '-3.5', '-3.5', '100', '3.49', '155', '28', '282.5', '292.5', '283.1']
['2021-12-16', 'Vernon', '700.0', '2884', '-14.3', '-22.3', '51', '0.92', '205', '13', '286.6', '289.5', '286.8']
['2021-12-16', 'Vernon', '850.0', '1392', '-6.7', '-10.5', '74', '2.03', '135', '9', '279.1', '285.1', '279.5']
['2021-12-16', 'Port Hardy', '700.0', '2874', '-10.7', '-13.7', '79', '1.91', '140', '9', '290.6', '296.5', '290.9']
['2021-12-16', 'Port Hardy', '850.0', '1332', '-2.5', '-6.9', '72', '2.69', '135', '14', '283.5', '291.4', '284.0']
['2023-06-20', 'Quillayute', '700.0', '2999', '-6.7', '-7.5', '94', '3.12', '305', '14', '295.0', '304.6', '295.6']
['2023-06-20', 'Quillayute', '850.0', '1455', '2.2', '2.2', '100', '5.30', '255', '20', '288.4', '303.7', '289.4']
['2023-06-20', 'Vernon', '700.0', '2977', '-5.9', '-6.3', '97', '3.43', '29

In [273]:
print("Problem URLs:")
for url in problem_urls:
    print(url)

Problem URLs:


## Clean the Extracted Data
---

In [274]:
# Remove rogue "e" values from pressure field
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
pressures = [700, 850]
ctr = 0
for item in data:
    if(not str(item[2]).isnumeric()):
        item[2] = pressures[ctr%2]
    ctr += 1

In [275]:
cols = ["Date", 
        "Station", 
        "Pressure", 
        "Height", 
        "Temp", 
        "DewPoint", 
        "Relative_Humidity", 
        "Mean_Mixed_Layer", 
        "Wind_Direction", 
        "Wind_Speed", 
        "Potential_Temp",
        "Equivalent_Potential_Temp",
        "Virtual_Potential_Temp"]

In [276]:
new_data = pd.DataFrame(data, columns=cols)

# Convert DataFrame to numeric values
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for col in cols[2:]:
    try:
        new_data[col] = pd.to_numeric(new_data[col])
    except:
        print(col)
    
# Convert wind speed from Knots to KM/H
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
new_data['Wind_Speed'] = new_data['Wind_Speed'].multiply(1.852)
new_data.head(10)

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
0,2021-12-16,Quillayute,700,2844,-10.9,-11.3,97,2.31,125,29.632,290.4,297.4,290.8
1,2021-12-16,Quillayute,850,1328,-3.5,-3.5,100,3.49,155,51.856,282.5,292.5,283.1
2,2021-12-16,Vernon,700,2884,-14.3,-22.3,51,0.92,205,24.076,286.6,289.5,286.8
3,2021-12-16,Vernon,850,1392,-6.7,-10.5,74,2.03,135,16.668,279.1,285.1,279.5
4,2021-12-16,Port Hardy,700,2874,-10.7,-13.7,79,1.91,140,16.668,290.6,296.5,290.9
5,2021-12-16,Port Hardy,850,1332,-2.5,-6.9,72,2.69,135,25.928,283.5,291.4,284.0
6,2023-06-20,Quillayute,700,2999,-6.7,-7.5,94,3.12,305,25.928,295.0,304.6,295.6
7,2023-06-20,Quillayute,850,1455,2.2,2.2,100,5.30,255,37.040,288.4,303.7,289.4
8,2023-06-20,Vernon,700,2977,-5.9,-6.3,97,3.43,290,12.964,295.9,306.4,296.5
9,2023-06-20,Vernon,850,1429,3.6,2.4,92,5.38,210,14.816,289.9,305.5,290.9


In [277]:
new_data.tail()

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
13,2023-06-21,Quillayute,850,1535,4.6,1.1,78,4.90,280,3.704,290.9,305.3,291.8
14,2023-06-21,Vernon,700,3075,-4.9,-5.8,93,3.56,355,22.224,297.0,307.9,297.7
15,2023-06-21,Vernon,850,1519,5.2,0.9,74,4.83,0,22.224,291.6,305.8,292.4
16,2023-06-21,Port Hardy,700,3105,-0.3,-34.3,6,0.30,60,33.336,302.1,303.2,302.2
17,2023-06-21,Port Hardy,850,1538,8.4,-0.6,53,4.33,40,5.556,294.9,307.9,295.7


## Combine New and Old Data

In [285]:
complete_data = pd.concat([weather_data, new_data])
complete_data = complete_data.sort_values(by=['Date'])
#complete_data = complete_data.drop('Unnamed: 0', axis=1)
complete_data.head(20)

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
0,2020-01-01,Quillayute,700,2969.0,-0.9,-0.9,100.0,5.14,255.0,100.008,301.5,317.2,302.4
1,2020-01-01,Quillayute,850,1393.0,6.2,6.2,100.0,7.04,260.0,81.488,292.6,313.1,293.9
2,2020-01-01,Vernon,700,2880.0,-5.5,-7.0,89.0,3.25,260.0,94.452,296.4,306.3,296.9
3,2020-01-01,Vernon,850,1357.0,-3.9,-5.8,87.0,2.93,170.0,18.520,282.1,290.5,282.6
4,2020-01-01,Port Hardy,700,2856.0,-4.3,-5.8,89.0,3.56,250.0,87.044,297.7,308.6,298.3
5,2020-01-01,Port Hardy,850,1301.0,3.4,1.9,90.0,5.19,250.0,59.264,289.7,304.8,290.6
6,2020-01-02,Quillayute,700,2946.0,-10.7,-23.7,34.0,0.81,260.0,50.004,290.6,293.2,290.8
7,2020-01-02,Quillayute,850,1426.0,-1.3,-1.6,98.0,4.02,280.0,42.596,284.8,296.4,285.5
8,2020-01-02,Vernon,700,2882.0,-9.5,-12.3,80.0,2.14,305.0,31.484,291.9,298.5,292.3
9,2020-01-02,Vernon,850,1351.0,2.2,-4.8,60.0,3.16,245.0,33.336,288.4,297.8,289.0


In [286]:
complete_data.tail(20)

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
7583,2023-06-18,Port Hardy,850,1460.0,-0.1,-1.1,93.0,4.17,260.0,38.892,286.0,298.1,286.8
7578,2023-06-18,Quillayute,700,3027.0,-4.7,-25.7,18.0,0.68,290.0,31.484,297.2,299.5,297.4
7584,2023-06-19,Quillayute,700,2957.0,-9.3,-12.9,75.0,2.03,290.0,20.372,292.2,298.4,292.5
7588,2023-06-19,Port Hardy,700,2964.0,-8.9,-20.9,37.0,1.04,60.0,14.816,292.6,295.9,292.8
7587,2023-06-19,Vernon,850,1404.0,5.8,-0.2,65.0,4.46,230.0,24.076,292.2,305.4,293.0
7586,2023-06-19,Vernon,700,2957.0,-6.3,-6.3,100.0,3.43,330.0,18.520,295.5,305.9,296.1
7585,2023-06-19,Quillayute,850,1430.0,0.0,-3.2,79.0,3.57,290.0,9.260,286.1,296.5,286.8
7589,2023-06-19,Port Hardy,850,1431.0,1.4,-4.6,64.0,3.21,55.0,7.408,287.6,297.1,288.2
11,2023-06-20,Port Hardy,850,1454.0,7.8,-0.2,57.0,4.46,20.0,24.076,294.3,307.6,295.1
10,2023-06-20,Port Hardy,700,3020.0,-4.1,-7.8,75.0,3.05,50.0,40.744,297.9,307.4,298.5


## Export Clean Data to CSV
---

In [290]:
# Save backup before overwriting weather.csv
filepath = Path('./data/backups/weather-{}.csv'.format(datetime.today().date()))
filepath.parent.mkdir(parents=True, exist_ok=True) 
weather_data.to_csv(filepath, index=False)

In [291]:
# Overwrite existing weather.csv with most recent data
filepath = Path('./data/weather.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 
complete_data.to_csv(filepath, index=False)